# Fetching data on university postcodes using an API

This notebook documents the process followed in the third session on Python, focusing on APIs.

First we import the `pandas` library which we'll need to import the data from the API.

In [ ]:
import pandas as pd

Our scenario is that we have data on universities but we need to be able to aggregate those universities into the particular region they are in. 

We also might want to query the police API for data on crimes - and for that we will need the lat/long of each university.

So we need to import the data on university postcodes that we are going to use to 'query' the postcodes API for the region and lat/long.

In [4]:
#import a CSV of university names and postcodes, shared from Google Drive
unidataurl = "https://docs.google.com/spreadsheets/d/e/2PACX-1vT9cEtSDCK6vrYxCIWv7Tg7fxCD8z3p_FoTj6b-3ipE4jYuH06ozsmz02oYqNoRHZlIGz5ECyfWZJbh/pub?output=csv"

In [6]:
#read in the CSV at that URL
unidata = pd.read_csv(unidataurl)
#preview it
unidata

,uni,postcode
0,Anglia Ruskin University,CM1 1SQ
1,Aston University,B4 7ET
2,Bangor University,LL57 2DG
3,Bath Spa University,BA2 9BN
4,Birkbeck College,WC1E 7HX
...,...,...
161,University of the Highlands and Islands,IV3 5SQ
162,"University of the West of England, Bristol",BS16 1QY
163,University of the West of Scotland,PA1 2BE
164,Writtle College,CM1 3RR


## Query the API

Now let's use that data to query the API, starting with one postcode as a test.

In [8]:
#grab the first postcode in our data
firstpostcode = unidata['postcode'][0]
firstpostcode

'CM1 1SQ'

In [19]:
#form the URL - an endpoint - for that
#start with the base URL that we need to add the postcode to
baseurl = "https://api.postcodes.io/postcodes/"
#remove any spaces
firstpostcode = firstpostcode.replace(' ','')
#create a full URL
fullurl = baseurl+firstpostcode
print(fullurl)
#fetch the json
json = pd.read_json(fullurl)
#drill down into it to grab the lat, long and region
lat = json['result']['latitude']
lng = json['result']['longitude']
region = json['result']['region']
#check
print(lat)
print(lng)
print(region)

https://api.postcodes.io/postcodes/CM11SQ
51.741381
0.474495
East of England


## Create a dataframe to hold the data we get - and loop to create rows for that dataframe

Now we've tested it on one postcode, we can create a loop to go through multiple postcodes. 

To begin with we can limit the loop to the first five items by adding the index 'slice' `[:5]` to the start of the loop like so:

`for i in unidata['postcode'][:5]:`

Then, once it's working on those 5, we can remove it so we loop through all items.

When we do extend it to all items, we will hit an error (a `HTTPError`) with at least one postcode (b422su) because the API doesn't have data for it.

To solve that problem, we add a `try/except` branch to the loop, which means it doesn't stop when it hits the error, but just does something else (`except:`) instead, storing the fact that for those postcodes there was no data.

In [ ]:
#create a dataframe - it's only got one column for now
apidata = pd.DataFrame(columns=['postcode'])

#loop through the postcodes - first 5 to begin with
for i in unidata['postcode']:
  #replace the space
  postcodeclean = i.replace(' ','')
  #check it worked
  print(postcodeclean)
  #create the url
  jsonurl = baseurl+postcodeclean
  #check
  print(jsonurl)
  #try the code after the colon - but if it breaks it won't stop the loop
  #instead it will skip to 'except' below
  #DON'T FORGET THE COLON!
  try:
    #fetch from url
    json = pd.read_json(jsonurl)
    #create new dictionary object
    newrow = {}
    #fill that dictionary under the key 'postcode'
    newrow['postcode'] = i
    #grab the lat, long and region
    lat = json['result']['latitude']
    lng = json['result']['longitude']
    region = json['result']['region']
    #add them to the dictionary
    newrow['latitude'] = lat
    newrow['longitude'] = lng
    newrow['region'] = region
    newrow['API URL'] = jsonurl
    #show the dictionary
    print(newrow)
    #add to the dataframe
    apidata = apidata.append(newrow, ignore_index=True)
  #except specifies what to do if the 'try' chunk throws an error
  #(for example because the page doesn't exist)
  #DON'T FORGET THE COLON!
  except:
    #create new dictionary object
    newrow = {}
    #fill it so we know the API doesn't have data for this postcode 
    newrow['postcode'] = i
    newrow['latitude'] = "NO DATA"
    newrow['longitude'] = "NO DATA"
    newrow['region'] = "NO DATA"
    #add to the dataframe
    apidata = apidata.append(newrow, ignore_index=True)

print(apidata)

In [42]:
apidata

,postcode,latitude,longitude,region
0,CM1 1SQ,51.7414,0.474495,East of England
1,B4 7ET,52.4866,-1.89095,West Midlands
2,LL57 2DG,53.2292,-4.12944,None
3,BA2 9BN,51.3732,-2.44091,South West
4,WC1E 7HX,51.5219,-0.130359,London
...,...,...,...,...
161,IV3 5SQ,57.4694,-4.23024,None
162,BS16 1QY,51.5011,-2.54746,South West
163,PA1 2BE,55.8435,-4.42997,None
164,CM1 3RR,51.7372,0.428133,East of England


In [43]:
#and export
apidata.to_csv("apidata.csv")

## Version where we keep the uni name too

In the loop above we only retain the postcodes from the original data, so we'd have to merge it back with the original data to get the university names as well. 

Alternatively, we could write our loop using indices instead, fetching both the postcode *and* the university name, and storing both in the resulting dataframe. 

First, here's some code demonstrating the concept of using indices instead:

In [47]:
#find out how many universities there are in our data
uninumbers = len(unidata)
#it's 166
print(uninumbers)
#create a range of indices from 0 to 165 (the 166th item), and loop through those
for i in range(0,uninumbers):
  print(i)
  #fetch the postcode at that index
  print(unidata['postcode'][i])
  #fetch the uni at that index
  print(unidata['uni'][i])

166
0
CM1 1SQ
Anglia Ruskin University
1
B4 7ET
Aston University
2
LL57 2DG
Bangor University
3
BA2 9BN
Bath Spa University
4
WC1E 7HX
Birkbeck College
5
B42 2SU
Birmingham City University
6
LN1 3DY
Bishop Grosseteste University College Lincoln
7
BH12 5BB
Bournemouth University
8
UB8 3PH
Brunel University
9
HP11 2JZ
Buckinghamshire New University
10
CT1 1UT
Canterbury Christ Church University
11
CF5 2SG
Cardiff Metropolitan University
12
CF10 3AT
Cardiff University
13
NW3 3HY
Central School of Speech and Drama(The)
14
WC2R 0RN
Courtauld Institute of Art
15
CV1 5FB
Coventry University
16
MK43 0AL
Cranfield University
17
LE1 9BH
De Montfort University
18
L39 4QP
Edge Hill University
19
EH11 4BN
Edinburgh Napier University
20
G4 0BA
Glasgow Caledonian University
21
G3 6RQ
Glasgow School of Art
22
LL11 2AW
Glyndwr University
23
SE14 6NW
Goldsmiths' College
24
EC2Y 8DT
Guildhall School of Music and Drama
25
TF10 8NB
Harper Adams University
26
EH14 4AS
Heriot-Watt University
27
W8 5HN
Heythr

Now this is what it looks like incorporated into a loop:

In [ ]:
#create a dataframe - it's only got one column for now
apidata = pd.DataFrame(columns=['postcode'])

uninumbers = len(unidata)

#loop through the postcodes - first 5 to begin with
for i in range(0,uninumbers):
  print("index", i)
  postcode = unidata['postcode'][i]
  #replace the space
  postcodeclean = postcode.replace(' ','')
  #check it worked
  print(postcodeclean)
  #create the url
  jsonurl = baseurl+postcodeclean
  #check
  print(jsonurl)
  #try the code after the colon - but if it breaks it won't stop the loop
  #instead it will skip to 'except' below
  #DON'T FORGET THE COLON!
  try:
    #fetch from url
    json = pd.read_json(jsonurl)
    #create new dictionary object
    newrow = {}
    #fill that dictionary under the key 'postcode'
    newrow['postcode'] = postcode
    #and the uni
    newrow['university'] = unidata['uni'][i]
    #grab the lat, long and region
    lat = json['result']['latitude']
    lng = json['result']['longitude']
    region = json['result']['region']
    #add them to the dictionary
    newrow['latitude'] = lat
    newrow['longitude'] = lng
    newrow['region'] = region
    newrow['API URL'] = jsonurl
    #show the dictionary
    print(newrow)
    #add to the dataframe
    apidata = apidata.append(newrow, ignore_index=True)
  #except specifies what to do if the 'try' chunk throws an error
  #(for example because the page doesn't exist)
  #DON'T FORGET THE COLON!
  except:
    #create new dictionary object
    newrow = {}
    #fill it so we know the API doesn't have data for this postcode 
    newrow['postcode'] = postcode
    #and the uni
    newrow['university'] = unidata['uni'][i]
    newrow['latitude'] = "NO DATA"
    newrow['longitude'] = "NO DATA"
    newrow['region'] = "NO DATA"
    #add to the dataframe
    apidata = apidata.append(newrow, ignore_index=True)

print(apidata)

index 0
CM11SQ
https://api.postcodes.io/postcodes/CM11SQ
{'postcode': 'CM1 1SQ', 'university': 'Anglia Ruskin University', 'latitude': 51.741381, 'longitude': 0.474495, 'region': 'East of England', 'API URL': 'https://api.postcodes.io/postcodes/CM11SQ'}
index 1
B47ET
https://api.postcodes.io/postcodes/B47ET
{'postcode': 'B4 7ET', 'university': 'Aston University', 'latitude': 52.486637, 'longitude': -1.890952, 'region': 'West Midlands', 'API URL': 'https://api.postcodes.io/postcodes/B47ET'}
index 2
LL572DG
https://api.postcodes.io/postcodes/LL572DG
{'postcode': 'LL57 2DG', 'university': 'Bangor University', 'latitude': 53.229193, 'longitude': -4.129437, 'region': None, 'API URL': 'https://api.postcodes.io/postcodes/LL572DG'}
index 3
BA29BN
https://api.postcodes.io/postcodes/BA29BN
{'postcode': 'BA2 9BN', 'university': 'Bath Spa University', 'latitude': 51.373209, 'longitude': -2.440912, 'region': 'South West', 'API URL': 'https://api.postcodes.io/postcodes/BA29BN'}
index 4
WC1E7HX
https